<a href="https://colab.research.google.com/github/engcarlosperezmolero/colabs_sharing/blob/main/group_and_agg_pandas_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Hack para visualizar mas comodamente 🤓


In [ ]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 20px;">
    <p style='font-family:"Courier New", Courier, monospace'><strong>{0}</strong></p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_()) for a in self.args)

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a)) for a in self.args)

In [ ]:
import pandas as pd

df1 = pd.DataFrame({"col1": [1,2,3,4], "col2": list("abcd")})
df2 = pd.DataFrame({"col3": [55,12,7668,31,123], "col4": ["aasd", "akjsdh", "lakjsdf", "kjasdlk", "lkzdj"]})

In [ ]:
display("df1", "df2")

---
# Acerca del dataset cammesa-renovable-energies 🧐


https://cammesaweb.cammesa.com/erenovables/

COMPAÑÍA ADMINISTRADORA DEL MERCADO MAYORISTA ELECTRICO SOCIEDAD ANÓNIMA (CAMMESA)

La Base de Datos contiene información relativa a la generación de energía eléctrica proveniente de fuentes renovables de energía en relación al RÉGIMEN DE FOMENTO NACIONAL PARA EL USO DE FUENTES RENOVABLES DE ENERGÍA DESTINADA A LA PRODUCCIÓN DE ENERGÍA ELÉCTRICA, sancionado a través de la Ley N° 26.190

La Base de Datos es de paso mensual, desde Enero 2011, conteniendo los siguientes campos:

a) Central / máquina por región del país;

b) Tipo de fuente de energía renovable (Hidro < 50 MW; Eólico, Solar, Biomasa, Biodiesel)

c) Energía Generada (GWh)

d) Demanda total MEM (GWh)

---
# Descargamos el dataset de kaggle 📊

In [ ]:
def download_kaggle_dataset_to_colab_instance(api_command_from_kaggle: str, json_content: str) -> None:
    """
    descarga un dataset de la base de datasets de kaggle y lo guarda en la instancia de google colab activa.

    args: 
        api_command_from_kaggle: es la linea de api que genera kaggle sobre el dataset, se consigue en las opciones del dataset.
        json_content: es el contenido del json que genera Kaggle cuando se genera una API TOKEN.
    
    """
    from zipfile import ZipFile
    ! pip install kaggle
    ! mkdir /root/.kaggle/
    with open("/root/.kaggle/kaggle.json", "w") as token:
        token.write(json_content)
    ! chmod 600 /root/.kaggle/kaggle.json
    ! kaggle config path -p /root/.kaggle/
    ! $api_command_from_kaggle
    name = f"{api_command_from_kaggle.split('/')[-1]}.zip"
    zip_ref = ZipFile(f"/content/{name}")
    zip_ref.extractall()
    zip_ref.close()
    ! rm $name


In [ ]:
download_kaggle_dataset_to_colab_instance("kaggle datasets download -d ccollado7/alternative-energies-argentina", '{"username":"charlymolero","key":"1d6b921759c70f37b628bb3d636a143a"}')

---
# Ahora usamos el dataset 😎

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_csv("/content/energias-alternativas.csv", encoding="unicode_escape", engine="python")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.head()

---
# Analicemos la Generación de energia para cada tipo de energia alternativa en argentina (suma).

In [ ]:
df.actividad_producto_nombre.value_counts()

In [ ]:
df.indicador.value_counts()

In [ ]:
df.unidad_de_medida.value_counts()

In [ ]:
generacion_por_energia = df[df["indicador"] == "Generación"].groupby("actividad_producto_nombre").aggregate({"valor": "sum"}).sort_values(by=["valor"], ascending=False).reset_index()

In [ ]:
generacion_por_energia

In [ ]:
fig = px.bar(x = generacion_por_energia.actividad_producto_nombre, y = generacion_por_energia.valor)
fig.update_xaxes(type='category')
fig.update_layout(
    xaxis_title="Energia Alternativa",
    yaxis_title="Generación (MWh)",
)
fig.show()

---
# Analicemos la Generación en cada provincia (suma).

In [ ]:
df.head()

In [ ]:
df[df['indicador'] == "Generación"].groupby("alcance_nombre").aggregate({"valor": "sum"})

In [ ]:
energia_por_provincia = df[df['indicador'] == "Generación"].groupby("alcance_nombre").aggregate({"valor": "sum"}).sort_values(by=["valor"]).reset_index()

In [ ]:
energia_por_provincia

In [ ]:
fig = px.bar(x = energia_por_provincia.alcance_nombre, y = energia_por_provincia.valor)
fig.update_xaxes(type='category')
fig.update_layout(
    xaxis_title="Provincia",
    yaxis_title="Generación (MWh)",
)
fig.show()

---
# Analicemos los tipos de energia en cada provincia (conteo y multiindex)

In [ ]:
df.head()

In [ ]:
df_prov = df[df["indicador"] == "Generación"].groupby(["alcance_nombre", "actividad_producto_nombre"]).aggregate({"valor": ["size", "sum"]})

---
# Analicemos la generación de energia por año y mes (suma, multindex, group con datetimes y unstack)

### usando la columna tal y como viene (en object -> str)

In [ ]:
df.head()

In [ ]:
df.indice_tiempo.str.split("-", expand = True)[1]

In [ ]:
df["year"] = df.indice_tiempo.str.split("-", expand = True)[0]
df["month"] = df.indice_tiempo.str.split("-", expand = True)[1]

In [ ]:
df[df["indicador"] == "Generación"].groupby(["year", "month"]).aggregate({"valor": "sum"})

### casteando a datetime

In [ ]:
dfdt = df.copy()
dfdt.info()

In [ ]:
pd.to_datetime(dfdt.indice_tiempo)


In [ ]:
dfdt.indice_tiempo.dt.month_name()

In [ ]:
dfdt[dfdt.indicador == "Generación"].groupby([dfdt.indice_tiempo.dt.year, dfdt.indice_tiempo.dt.month_name()]).aggregate({"valor": "sum"}).unstack()

---
# El dinamismo entre pivot_table y multi index groupby. ¿Dos caras de una moneda? 🤔
## Analicemos como es la generacion de cada tipo de energia por año

In [ ]:
df.head()

In [ ]:
df[df["indicador"] == "Generación"].pivot_table(
    index = ["actividad_producto_nombre", "year"],
    columns = [],
    values = ["valor"],
    aggfunc = ["sum"],
    margins = True
).unstack()

In [ ]:
# con stack() podemos generar una tabla agrupada a partir de una pivot table on the fly
energia_anio_valor_pivot_stack = df[df["indicador"] == "Generación"].pivot_table(
    index = ["actividad_producto_nombre"],
    columns = ["year"],
    values = ["valor"],
    aggfunc = ["sum"],
    margins = False
).stack()

In [ ]:
# una serie con dos dimensiones ["actividad_producto_nombre", "year"] puede ser representada como un dataframe usando el metodo unstack()
energia_anio_valor_group = df[df["indicador"] == "Generación"].groupby(["actividad_producto_nombre", "year"]).aggregate({"valor": "sum"})

<p>
<q style='quotes: "“" "”";'>Many programmers and SQL analysts ﬁnd the .groupby syntax intuitive,<br>while Excel junkies often feel more at home with the .pivot_table method.</q>
</p>

*Effective Pandas: Patterns For Data Manipulation, Matt Harrison (2021) p. 240*

In [ ]:
display("energia_anio_valor_pivot_stack", "energia_anio_valor_group")

#### groupby y pivot admiten especificar funciones para cada columna por separado

In [ ]:
df_pivot_specific = df[df["indicador"] == "Generación"].pivot_table(
                                                index = "actividad_producto_nombre",
                                                aggfunc = {
                                                            "sector_id": ["size"],
                                                            "valor": ["min", "max"]
                                                }
)

In [ ]:
df_group_specific = df[df["indicador"] == "Generación"].groupby("actividad_producto_nombre").aggregate({
                                                                            "sector_id": ["size"],
                                                                            "valor": ["min", "max"]    
})

In [ ]:
display("df_pivot_specific", "df_group_specific")

---
# Bonus tricks 🔥💥⚡

### np.where



```python
np.where(
    condicional (mascara booleana),
    serie/lista/funcion()/escalar si es True,
    serie/lista/funcion()/escalar si es False
)
```
usar el atributo .values en las series del df que se usen:


```python
df["nombre_columna"].values
```





### np.select (para multiples condiciones)



```python
conditions = [lista de mascaras booleanas]

choices = [lista de lo que se coloca en caso de que la mascara booleana sea True]

np.select(conditions, choices, default="NA")
```
prestar atencion a que el argumento **default** es el valor que toma en caso de que ninguna de las condiciones booleanas sea True.


In [ ]:
conditions = [
            dfdt.indice_tiempo.dt.month.isin([1, 2, 3]), # verano
            dfdt.indice_tiempo.dt.month.isin([4, 5, 6]), # otoño
            dfdt.indice_tiempo.dt.month.isin([7, 8, 9]), # invierno
            dfdt.indice_tiempo.dt.month.isin([10, 11, 12]),  # primavera
]

choices = [
           "verano",
           "otoño",
           "invierno",
           "primavera"
]

dfdt["estacion"] = np.select(conditions, choices, default="no hay info")

In [ ]:
dfdt_season_np_select = dfdt.groupby("estacion").aggregate({"valor": ["size", "count", "sum"]}) # count retorna el conteo de valores no NaN.

In [ ]:
dfdt_season_np_select

#### Tambien podemos lograr el mismo output usando agrupamiento con funcion sobre el index

In [ ]:
dfdt_index = dfdt.set_index("indice_tiempo")

In [ ]:
def season_grouper(idx):
    if idx.month in [1, 2, 3]:
        return "verano"
    elif idx.month in [4, 5, 6]:
        return "otoño"
    elif idx.month in [7, 8, 9]:
        return "invierno"
    elif idx.month in [10, 11, 12]:
        return "primavera"
    else:
        return "no hay info"

In [ ]:
dfdt_index_pivot = dfdt_index.pivot_table(index=season_grouper, values=["valor"], aggfunc=["size", "count", "sum"])



dfdt_index_group = dfdt_index.groupby(season_grouper).aggregate({"valor": ["size", "count", "sum"]})

In [ ]:
display("dfdt_index_pivot", "dfdt_index_group", "dfdt_season_np_select")

In [ ]:
def funcion_personalizada(x):
    return x.mean() * 100

df.groupby("actividad_producto_nombre").aggregate({"valor": funcion_personalizada})

In [ ]:
df.groupby("actividad_producto_nombre").aggregate({"valor": "mean"}) * 100

### seleccion con multi index: la magia del .loc[ ]




```python
df.loc[filas, columnas]

df.loc[(niveles de indice separados por coma), :]

slice(None) se usa para seleccionar todos indices de un nivel dado

```

### strip para los nombres de columnas

In [ ]:
test.columns = test.columns.str.strip()

In [ ]:
test.columns = [col.strip() for col in test.columns]